In [1]:
import os
import re
import ast
import numpy as np
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

from trulens_eval import Tru
from trulens_eval import TruCustomApp
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.langchain import Langchain
from trulens_eval.tru_custom_app import instrument
from trulens_eval.feedback import prompts
import custom_prompts

from langchain_ibm import WatsonxLLM
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

from genai import Client, Credentials
from genai.extensions.langchain import LangChainInterface
from genai.schema import (
    DecodingMethod,
    TextGenerationParameters,
)

load_dotenv()

True

In [2]:
def bam_model(model_id='mistralai/mixtral-8x7b-instruct-v0-1', decoding_method='greedy', max_new_tokens=1000, 
              min_new_tokens=1, temperature=0.5, top_k=50, top_p=1, repetition_penalty=1):

    if decoding_method == 'greedy':
        decoding_method = DecodingMethod.GREEDY
        parameters=TextGenerationParameters(
            decoding_method=decoding_method,
            max_new_tokens=max_new_tokens,
            min_new_tokens=min_new_tokens,
            repetition_penalty=repetition_penalty
        )
    else:
        decoding_method = DecodingMethod.SAMPLE
        parameters=TextGenerationParameters(
            decoding_method=decoding_method,
            max_new_tokens=max_new_tokens,
            min_new_tokens=min_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            repetition_penalty=repetition_penalty
        )

    llm = LangChainInterface(
        model_id=model_id,
        client=Client(credentials=Credentials.from_env()),
        parameters=parameters,
    )

    return llm

In [3]:
mixtral_llm = bam_model(model_id="mistralai/mixtral-8x7b-instruct-v0-1", repetition_penalty=1.1)

prompt = "Tell me about IBM."
result = mixtral_llm.invoke(prompt)
print(result)


IBM is a global technology and innovation company headquartered in Armonk, NY. It is the largest technology employer in the world, with more than 375,000 employees serving clients in 170 countries. IBM offers a wide range of technology and consulting services; a broad portfolio of middleware for collaboration, predictive analytics, software development and systems management; and the world's most advanced servers and supercomputers. Utilizing its business consulting, technology and R&D expertise, IBM helps clients become "smarter" as the planet becomes more digitally interconnected. IBM invests more than $6 billion a year in R&D, just completing its 20th consecutive year of patent leadership. IBM Research has received recognition beyond any commercial technology research organization and is home to 5 Nobel Laureates, 9 US National Medals of Technology, 5 US National Medals of Science, 6 Turing Awards, and 10 Inductees in US Inventors Hall of Fame. The company was founded in 1911, and 

In [4]:
df = pd.read_csv("data/ms-marco.csv")
df["contexts"] = [ast.literal_eval(ctx) for ctx in df["contexts"]]
df

,question,contexts
0,walgreens store sales average,[The average Walgreens salary ranges from appr...
1,how much do bartenders make,[A bartender’s income is comprised mostly of t...
2,what is a furuncle boil,"[Knowledge center. A boil, also known as a fur..."
3,what can urinalysis detect,[Urinalysis: One way to test for bladder cance...
4,what is vitamin a used for,[Since vitamin A is fat-soluble it is not need...
...,...,...
195,who wrote Nothing compares to you,[Nothing Compares 2 U is a song originally wr...
196,cost for relining dentures,[When dentures that used to fit are now loose ...
197,what is sherry wine made of,"[Sherry vinegar is made from sherry, a fortifi..."
198,dna in bacteria,[Bacterial DNA in Human Genomes. A new study f...


In [5]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# data = text_splitter.create_documents(df["contexts"].to_list(), metadatas=df[["question"]].to_dict(orient="records"))

def text_to_chunks(texts: str,
                   chunk_length: int = 100,
                   chunk_overlap: int = 25) -> list:
    """
    Splits the text into equally distributed chunks with 25-word overlap.
    Args:
        texts (str): Text to be converted into chunks.
        chunk_length (int): Maximum number of words in each chunk.
        chunk_overlap (int): Number of words to overlap between chunks.
    """
    words = texts.split(' ')
    n = len(words)
    chunks = []
    chunk_number = 1
    i = 0
    while i < n:  # Corrected the length check
        chunk = words[i: min(i + chunk_length, n)]
        i = i + chunk_length - chunk_overlap
        #print(len(chunk))
        chunk = ' '.join(chunk).strip()
        chunks.append({"text": chunk})
        chunk_number += 1
    return chunks

In [6]:
df['chunks'] = df['contexts'].apply(lambda x: [i['text'] for i in text_to_chunks(x[0])])
df['chunks']

0      [The average Walgreens salary ranges from appr...
1      [A bartender’s income is comprised mostly of t...
2      [Knowledge center. A boil, also known as a fur...
3      [Urinalysis: One way to test for bladder cance...
4      [Since vitamin A is fat-soluble it is not need...
                             ...                        
195    [Nothing Compares 2 U  is a song originally wr...
196    [When dentures that used to fit are now loose ...
197    [Sherry vinegar is made from sherry, a fortifi...
198    [Bacterial DNA in Human Genomes. A new study f...
199    [The movement of goods and services through th...
Name: chunks, Length: 200, dtype: object

In [7]:
data = []
for i in range(len(df)):
    for j in df['chunks'][i]:
        doc = Document(
            metadata={
                "question": df['question'][i],
            },
            page_content=j)
        data.append(doc)
data[0]

Document(page_content='The average Walgreens salary ranges from approximately $15,000 per year for Customer Service Associate / Cashier to $179,900 per year for District Manager. Average Walgreens hourly pay ranges from approximately $7.35 per hour for Laboratory Technician to $68.90 per hour for Pharmacy Manager. Salary information comes from 7,810 data points collected directly from employees, users, and jobs on Indeed.The average revenue in 2011 of a Starbuck Store was $1,078,000, up  from $1,011,000 in 2010.    The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.', metadata={'question': 'walgreens store sales average'})

In [8]:
# Initialie the embedding model
model_name = "intfloat/e5-large-v2"
model_kwargs = {'device': 'cpu'}

embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs
)

In [9]:
vectorstore = FAISS.from_documents(data, embeddings_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}, verbose = False)

In [10]:
def prompt_generation(context, query):
    template = (
        "<s>"
        "[INST] \n"
        "Context: {context} \n"
        "- Take the context above and use that to answer questions in a detailed and professional way. \n"
        "- If you don't know the answer just say \"I don't know\".\n"
        "- Refrain from using any other knowledge other than the text provided.\n"
        "- Don't mention that you are answering from the text, impersonate as if this is coming from your knowledge\n"
        "- For the questions whose answer is not available in the provided context, just say \"I don't know\".\n"
        "Question: {query}? \n"
        "[/INST] \n"
        "</s>\n"
        "Answer: "
    )

    qa_template = PromptTemplate.from_template(template)
    return qa_template.format(context=context, query=query)


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [11]:
query= "how much do bartenders make"
docs = retriever.get_relevant_documents(query)

pretty_print_docs(docs)

Document 1:

hour between their wages and tips. According to bartending.org, bartenders in a high volume resort or establishment can earn $50,000 to $75,000 per year between hourly wages and tips. Indeed.com 2010 results show bartenders in restaurants at median salary rates can make a good salary per year: Bartender $73,000.
----------------------------------------------------------------------------------------------------
Document 2:

a bartender make in one year. If the service is good and the coversation is i … nteresting a bartender can make alot of money. Location is also a big factor.Best Answer: An average bartender makes about...2 to 3 dollars an hour. but all the money is made off tips depending on the popularity of the bar. I used to make $700 to $1000 a night. but that is in Atlanta. If the bar is busy and you are a good bartender you will make quite a bit. I dont know how much, because I live in a town with
------------------------------------------------------------------

In [12]:
context = "\n".join([doc.page_content for doc in docs])
prompt = prompt_generation(context, query)
print(prompt)

<s>[INST] 
Context: hour between their wages and tips. According to bartending.org, bartenders in a high volume resort or establishment can earn $50,000 to $75,000 per year between hourly wages and tips. Indeed.com 2010 results show bartenders in restaurants at median salary rates can make a good salary per year: Bartender $73,000.
a bartender make in one year. If the service is good and the coversation is i … nteresting a bartender can make alot of money. Location is also a big factor.Best Answer: An average bartender makes about...2 to 3 dollars an hour. but all the money is made off tips depending on the popularity of the bar. I used to make $700 to $1000 a night. but that is in Atlanta. If the bar is busy and you are a good bartender you will make quite a bit. I dont know how much, because I live in a town with
busy and you are a good bartender you will make quite a bit. I dont know how much, because I live in a town with a population of 2000 so there is not alot going on around he

In [13]:
result = mixtral_llm.invoke(prompt)
print(f"Answer: {result}")

Answer: 
Bartenders can make varying amounts of money based on several factors such as location, hourly wages, and tips earned. In high-volume resorts or establishments, bartenders can earn between $50,000 to $75,000 per year, including both hourly wages and tips. However, in restaurants, the median salary rate for bartenders can be around $73,000 per year. On average, bartenders make about $2 to $3 an hour in wages, but they mostly rely on tips, which can range from $20 to $300 or more per night. The amount of money a bartender can make ultimately depends on various factors, including the popularity of the bar, the quality of service, and the level of conversation they provide to customers.

Based on the information provided, it seems that bartenders in smaller towns may earn less than those working in larger cities or popular tourist destinations. For instance, in a town with a population of 2000, a bartender might earn a total of $100 to $200 per night (on a good night), depending o

In [14]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)
prompt_rag_fusion

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are a helpful assistant that generates multiple search queries based on a single input query. \n\nGenerate multiple search queries related to: {question} \n\nOutput (4 queries):'))])

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain.load import dumps, loads

generate_queries = (
    prompt_rag_fusion 
    | mixtral_llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

In [16]:
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": query})
docs

/Users/sourav/workstuffs/env_exp/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


[(Document(page_content='hour between their wages and tips. According to bartending.org, bartenders in a high volume resort or establishment can earn $50,000 to $75,000 per year between hourly wages and tips. Indeed.com 2010 results show bartenders in restaurants at median salary rates can make a good salary per year: Bartender $73,000.', metadata={'question': 'how much do bartenders make'}),
  0.06639344262295083),
 (Document(page_content='A bartender’s income is comprised mostly of tips– 55% to be exact. In some states, employers aren’t even required to pay their bartenders the minimum wage and can pay as low as $2.13 per hour, and they depend on their tips almost entirely. Bartending can be a lot of things. For some it is exciting, for others exhausting. At times there is a lot of fun to be had, at others it is rather dull. But for the most part, bartending is almost always rewarding in the financial sense, as long as you stick with it.According to the Bureau of', metadata={'questio

In [17]:
retriever.get_relevant_documents(query)

[Document(page_content='hour between their wages and tips. According to bartending.org, bartenders in a high volume resort or establishment can earn $50,000 to $75,000 per year between hourly wages and tips. Indeed.com 2010 results show bartenders in restaurants at median salary rates can make a good salary per year: Bartender $73,000.', metadata={'question': 'how much do bartenders make'}),
 Document(page_content='a bartender make in one year. If the service is good and the coversation is i … nteresting a bartender can make alot of money. Location is also a big factor.Best Answer: An average bartender makes about...2 to 3 dollars an hour. but all the money is made off tips depending on the popularity of the bar. I used to make $700 to $1000 a night. but that is in Atlanta. If the bar is busy and you are a good bartender you will make quite a bit. I dont know how much, because I live in a town with', metadata={'question': 'how much do bartenders make'}),
 Document(page_content='busy an

In [18]:
[doc[0].page_content for doc in docs[:3]]

['hour between their wages and tips. According to bartending.org, bartenders in a high volume resort or establishment can earn $50,000 to $75,000 per year between hourly wages and tips. Indeed.com 2010 results show bartenders in restaurants at median salary rates can make a good salary per year: Bartender $73,000.',
 'A bartender’s income is comprised mostly of tips– 55% to be exact. In some states, employers aren’t even required to pay their bartenders the minimum wage and can pay as low as $2.13 per hour, and they depend on their tips almost entirely. Bartending can be a lot of things. For some it is exciting, for others exhausting. At times there is a lot of fun to be had, at others it is rather dull. But for the most part, bartending is almost always rewarding in the financial sense, as long as you stick with it.According to the Bureau of',
 "$18,970 a year. The median earnings of bartenders during this period were $9.06 an hour and $18,850 a year. Eighty percent of bartenders in t

In [19]:
class Processor:
    def __init__(self, retriever, llm):
        self._retriever = retriever
        self._llm = llm
    
    @instrument
    def retrieve_chunks(self, query, num_chunks):
        docs = self._retriever.invoke({"question": query})
        docs = [doc[0].page_content for doc in docs[:num_chunks]]
        return docs
    
    @instrument
    def join_chunks(self, chunks):
        return "\n".join(chunks)
    
    @instrument
    def respond_to_query(self, query, num_chunks=3):
        chunks = self.retrieve_chunks(query, num_chunks=num_chunks)
        context = self.join_chunks(chunks)
        prompt = prompt_generation(context, query)
        retries_left = 3
        while True:
            try:
                answer = self._llm.invoke(prompt).strip()
                break
            except Exception as e:
                print("Error while generating answer", e)
                if retries_left>0:
                    retries_left -= 1
                    print("Retrying. Retries Remaining -", retries_left)
                else:
                    raise
        return answer
    

In [20]:
rag_fusion = Processor(retrieval_chain_rag_fusion, mixtral_llm)

In [21]:
class IBMLangchain(Langchain):
    def _create_chat_completion(self, prompt = None, messages = None, **kwargs):
        if prompt is not None:
            # prompt += "\nANSWER:\n"
            prompt = f"[INST]\n{prompt}\n[/INST]"
            predict = self.endpoint.chain.invoke(prompt, **kwargs)
            predict = re.sub(r'Score: (\d+)/\d+', r'Score: \1', predict)

        elif messages is not None:
            prompt = messages[0]['content']
            # prompt += "\nANSWER:\n"
            prompt = f"[INST]\n{prompt}\n[/INST]"
            predict = self.endpoint.chain.invoke(prompt, **kwargs)
            predict = re.sub(r'Score: (\d+)/\d+', r'Score: \1', predict)

        else:
            raise ValueError("`prompt` or `messages` must be specified.")
        
        return predict
    
    def _groundedness_doc_in_out(self, premise: str, hypothesis: str) -> str:
        """
        An LLM prompt using the entire document for premise and entire statement
        document for hypothesis.

        Args:
            premise (str): A source document
            hypothesis (str): A statement to check

        Returns:
            str: An LLM response using a scorecard template
        """
        assert self.endpoint is not None, "Endpoint is not set."

        return self.endpoint.run_in_pace(
            func=self._create_chat_completion,
            prompt=str.format(custom_prompts.LLM_GROUNDEDNESS_FULL_SYSTEM,) +
            str.format(
                prompts.LLM_GROUNDEDNESS_FULL_PROMPT,
                premise=premise,
                hypothesis=hypothesis
            )
        )

In [22]:
eval_llm = bam_model(model_id="mistralai/mixtral-8x7b-instruct-v0-1", min_new_tokens=1, max_new_tokens=1000, repetition_penalty=1.1)
# eval_llm = bam_model(model_id="meta-llama/llama-2-70b-chat", max_new_tokens=1000, repetition_penalty=1.1)

langchain_provider = IBMLangchain(chain=eval_llm)

In [27]:
tru = Tru()
tru.reset_database()

In [28]:
# Question/statement relevance between question and each context chunk.
f_qs_relevance = (
    Feedback(
        langchain_provider.qs_relevance_with_cot_reasons,
        name="Context Relevance"
    )
    .on_input()
    .on(Select.RecordCalls.retrieve_chunks.rets[:])
    .aggregate(np.mean)
)

# Define a groundedness feedback function
grounded = Groundedness(groundedness_provider=langchain_provider)
f_groundedness = (
    Feedback(
        grounded.groundedness_measure_with_cot_reasons,
        name="Groundedness"
    )
    .on(Select.RecordCalls.join_chunks.rets) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(
    langchain_provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input().on_output()

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.retrieve_chunks.rets[:] .
✅ In Groundedness, input source will be set to __record__.app.join_chunks.rets .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [29]:
tru_rag = TruCustomApp(rag_fusion,
    app_id = 'RAG Fusion Pipeline',
    feedbacks = [f_qs_relevance, f_groundedness, f_qa_relevance])

In [31]:
with tru_rag as recording:
    for query in tqdm(df["question"], total=len(df)):
        ans = rag_fusion.respond_to_query(query)

100%|██████████| 200/200 [35:08<00:00, 10.54s/it]


In [32]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
RAG Fusion Pipeline,0.790167,0.737284,0.9915,9.855,0.0
